In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from PIL import Image
import numpy as np
import pandas as pd

In [3]:
import os

In [4]:
train_path='/content/drive/My Drive/MURA-v1.1/train/'
valid_path='/content/drive/My Drive/MURA-v1.1/valid/'

In [5]:
os.listdir()

['.config', 'drive', 'sample_data']

In [6]:
train_img_path=pd.read_csv('/content/drive/My Drive/MURA-v1.1/train_image_paths.csv')
valid_img_path=pd.read_csv('/content/drive/My Drive/MURA-v1.1/valid_image_paths.csv')
train_labels=pd.read_csv('/content/drive/My Drive/MURA-v1.1/train_labeled_studies.csv')
valid_labels=pd.read_csv('/content/drive/My Drive/MURA-v1.1/valid_labeled_studies.csv')

In [7]:
train_img_path.values[0]

array(['MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image2.png'],
      dtype=object)

In [8]:
train_labels.values[0]

array(['MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/', 1],
      dtype=object)

In [9]:
train_img_path.shape,train_labels.shape

((36807, 1), (13456, 2))

In [10]:
train_img_path.values[:5]

array([['MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image2.png'],
       ['MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image3.png'],
       ['MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image1.png'],
       ['MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image2.png'],
       ['MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image3.png']],
      dtype=object)

In [15]:
train_img_path['Label']=train_img_path.Img_Path.apply(lambda x:1 if 'positive' in x else 0)


AttributeError: ignored

In [16]:
valid_img_path.shape,valid_labels.shape

((3196, 1), (1198, 2))

In [17]:
valid_img_path['Label']=valid_img_path.Img_Path.apply(lambda x:1 if 'positive' in x else 0)

AttributeError: ignored

In [18]:
import torch
import torch.functional as F
from torch.utils.data import DataLoader,Dataset
from torchvision.transforms import Normalize,CenterCrop,Resize,Compose

In [19]:
class Mura(Dataset):
    
    def __init__(self,df,root,transform=None):
        self.df=df
        self.root=root
        self.transform=transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        img_name=self.df.iloc[idx,0]
        img=Image.open(img_name)
        label=self.df.iloc[idx,1]
        if transform:
            img=self.transform(img)
        return img,label

In [20]:
train_mura_dataset=Mura(df=train_img_path,root='../')

In [21]:
from torchvision import transforms

In [22]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [23]:
train_mura_dataset=Mura(df=train_img_path,root='./',transform=data_transforms['train'])

In [24]:
val_mura_dataset=Mura(df=valid_img_path,root='./',transform=data_transforms['val'])

In [25]:
train_loader=DataLoader(dataset=train_mura_dataset,batch_size=64,num_workers=4,shuffle=True)

In [26]:
val_loader=DataLoader(dataset=val_mura_dataset,batch_size=64,num_workers=4,shuffle=True)

In [27]:
dataloaders={
    'train':train_loader,
    'val':val_loader
}

In [28]:
dataset_sizes={
    'train':len(train_mura_dataset),
    'val':len(val_mura_dataset)
}

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

In [29]:
torch.cuda.is_available()

False

In [30]:
from sklearn.metrics import accuracy_score

In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [32]:
def train_model(model,criterion,optimizer,scheduler,num_epochs=1):
    start=time.time()
    
    best_model_wt=copy.deepcopy(model.state_dict())
    best_acc=0.0
    
    for epoch in range(num_epochs):
        print ('Epoch: {}/{}'.format(epoch+1,num_epochs))
        print ('-'*10)
        
        for phase in ['train','val']:
            if phase=='train':
                scheduler.step()
                model.train()
            else:
                model.eval()
            
            running_loss=0.0
            running_corrects=0
            
            for imgs,labels in dataloaders[phase]:
                imgs=imgs.to(device)
                labels=labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase=='train'):
                    outputs=model(imgs)
                    _,preds=torch.max(outputs)
                    loss=criterion(outputs,labels)
                    
                    if phase=='train':
                        loss.backward()
                        optimizer.step()
                
                running_loss+=loss.item()*imgs.size[0]
                running_corrects+=torch.sum(preds==labels)
                
                
            epoch_loss=running_loss/dataset_size[phase]
            epoch_acc=running_corrects.double()/dataset_size[phase]
            
            print ('{} Loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wt = copy.deepcopy(model.state_dict())
            
            print ()
            
        end=time.time()
        
        print('Training complete in {:.0f}m {:.0f}s'.format(
        (end-start) // 60, (end-start) % 60))
        print('Best val Acc: {:4f}'.format(best_acc))
        
        
        model.load_state_dict(best_model_wt)
        return model

In [35]:
base_model = torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=True)

#base_model=model.densenet169(pretrained=True)

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


In [36]:
for param in base_model.parameters():
    param.requires_grad = False


In [37]:
in_feat=base_model.classifier.in_features

In [38]:
model=nn.Sequential(
    base_model,
    nn.Linear(in_feat,1),
)

NameError: ignored

In [39]:
model=model.to(device)

NameError: ignored

In [ ]:
criterion=nn.BCELoss()

In [ ]:
optimizer=optim.Adam(params=model.parameters(),lr=0.01)

In [ ]:
exp_lr_scheduler=lr_scheduler.StepLR(optimizer,step_size=5,gamma=0.3)

In [ ]:
model=train_model(model,criterion,optimizer,exp_lr_scheduler)